In [26]:
import pandas as pd

In [27]:
raw_data_actions = pd.read_csv('users_hands_all_states_df.csv')

In [28]:
raw_data_actions = raw_data_actions.loc[raw_data_actions['user_action_type'].isin(['raises', 'bets', 'folds', 'checks', 'calls'])].copy()

In [29]:
raw_data_actions

,unique_id,user_step_num,hand_id,user_id,user_seat_num,flop11,flop12,flop21,flop22,flop31,...,turn_player_10_money_had_2,river_player_10_action_type_1,river_player_10_action_sum_1,river_player_10_money_had_1,river_player_10_action_type_2,river_player_10_action_sum_2,river_player_10_money_had_2,user_action_type,user_action_sum,user_result
0,SMGVLR_peperonexxx_5586_#000911458930:_1,1,#000911458930:,SMGVLR_peperonexxx,2,NaN,NaN,NaN,NaN,NaN,...,-1,unknown,-1,-1,unknown,-1,-1,raises,0.16,0.36
1,SMGVLR_peperonexxx_5586_#000911458930:_5,5,#000911458930:,SMGVLR_peperonexxx,2,Q,s,T,s,A,...,-1,unknown,-1,-1,unknown,-1,-1,bets,0.16,0.36
2,SMGVLR_peperonexxx_5586_#000911458930:_7,7,#000911458930:,SMGVLR_peperonexxx,2,Q,s,T,s,A,...,-1,unknown,-1,-1,unknown,-1,-1,bets,0.22,0.36
3,SMGVLR_peperonexxx_5586_#000911458971:_4,4,#000911458971:,SMGVLR_peperonexxx,2,NaN,NaN,NaN,NaN,NaN,...,-1,unknown,-1,-1,unknown,-1,-1,folds,0.00,-0.05
4,SMGVLR_peperonexxx_5586_#000911458988:_3,3,#000911458988:,SMGVLR_peperonexxx,2,NaN,NaN,NaN,NaN,NaN,...,-1,unknown,-1,-1,unknown,-1,-1,folds,0.00,-0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,muskoll_6626_#000911521913:_4,4,#000911521913:,muskoll,2,NaN,NaN,NaN,NaN,NaN,...,-1,unknown,-1,-1,unknown,-1,-1,calls,0.05,0.42
15857,muskoll_6626_#000911521913:_6,6,#000911521913:,muskoll,2,K,d,2,d,5,...,-1,unknown,-1,-1,unknown,-1,-1,raises,0.36,0.42
15858,muskoll_6626_#000911521913:_9,9,#000911521913:,muskoll,2,K,d,2,d,5,...,-1,unknown,-1,-1,unknown,-1,-1,bets,0.60,0.42
15859,parrukker__6609_#000911521916:_1,1,#000911521916:,parrukker_,2,NaN,NaN,NaN,NaN,NaN,...,-1,unknown,-1,-1,unknown,-1,-1,raises,0.06,0.03


In [30]:
raw_data_actions['user_step_num'].unique()

array([ 1,  5,  7,  4,  3,  6,  9, 11,  2,  8, 12, 10, 16, 19, 14, 15, 17,
       13, 18, 20, 23, 21, 22], dtype=int64)

In [31]:
def restructure(gdf):
        gdf.columns = ['_'.join(col).strip() for col in gdf.columns.values]
        gdf = gdf.stack().reset_index()
    
        b_d = {}
        for row in gdf.itertuples():
            b_d[row[1] + '_' + row[2] + '_' + row[3]] = row[-1]
        
        return b_d

    
def create_empty_d(column_name):
        b_d = {}
        for stage in ['pre-flop', 'flop', 'turn', 'river']:
            for action_type in ['raises', 'bets', 'folds', 'checks', 'calls']:
                for u in [f'{column_name}_count', f'{column_name}_mean', f'{column_name}_sum', f'{column_name}_std']:
                    b_d[stage + '_' + action_type + '_' + u] = 0
        return b_d

In [32]:
common_user_dicts = []

count = 0
for user in raw_data_actions.groupby('user_id'):
    big_user_df = user[1]
    big_user_df.sort_values(by=['unique_id'], inplace=True)
    print(count)
    
    for i in range(len(big_user_df)):
        user_df = big_user_df.iloc[:i].copy()
        unique_id = big_user_df['unique_id'].iloc[i]
    
        a = user_df['user_action_type'].value_counts()

        a_d = {'calls': 0, 'folds': 0, 'raises': 0, 'bets': 0, 'checks': 0}

        a_d.update(a.to_dict())

        b = user_df[['user_action_sum', 'stage', 'user_action_type']].groupby(['stage', 'user_action_type']).agg(['count', 'mean', 'sum', 'std'])

        b_d = create_empty_d('user_action_sum')
        b_d.update(restructure(b))

        c = user_df[['bank_before', 'stage', 'user_action_type']].groupby(['stage', 'user_action_type']).agg(['count', 'mean', 'sum', 'std'])

        c_d = create_empty_d('bank_before')
        c_d.update(restructure(c))

        d = user_df[['money_i_have', 'stage', 'user_action_type']].groupby(['stage', 'user_action_type']).agg(['count', 'mean', 'sum', 'std'])

        d_d = create_empty_d('money_i_have')
        d_d.update(restructure(d))


        user_df['div_action_sum_bank'] = user_df['user_action_sum'] - user_df['bank_before']
        e = user_df[['div_action_sum_bank', 'stage', 'user_action_type']].groupby(['stage', 'user_action_type']).agg(['count', 'mean', 'sum', 'std'])

        e_d = create_empty_d('div_action_sum_bank')
        e_d.update(restructure(e))

        user_df['div_action_sum_money_have'] = user_df['user_action_sum'] - user_df['money_i_have']
        f = user_df[['div_action_sum_money_have', 'stage', 'user_action_type']].groupby(['stage', 'user_action_type']).agg(['count', 'mean', 'sum', 'std'])

        f_d = create_empty_d('div_action_sum_money_have')
        f_d.update(restructure(f))

        g = user_df[['user_action_sum', 'user_step_num', 'stage', 'user_action_type']].groupby(['user_step_num', 'stage', 'user_action_type']).agg(['count', 'mean', 'sum', 'std'])


        g_d = {}

        for i in range(1, 25):
            for stage in ['pre-flop', 'flop', 'turn', 'river']:
                for action_type in ['raises', 'bets', 'folds', 'checks', 'calls']:
                    for u in [f'user_action_sum_count', f'user_action_sum_mean', f'user_action_sum_sum', f'user_action_sum_std']:
                        g_d[str(i) + '_' + stage + '_' + action_type + '_' + u] = 0

        g.columns = ['_'.join(col).strip() for col in g.columns.values]
        g = g.stack().reset_index()
        for row in g.itertuples():
            g_d[str(row[1]) + '_' + row[2] + '_' + row[3] + '_' + row[4]] = row[-1]

        common_dict = {**a_d, **b_d, **c_d, **d_d, **e_d, **f_d, **g_d, **{'user_id': user[0]}, **{'unique_id': unique_id}}

        common_user_dicts.append(common_dict)
    
    count += 1
manual_embeddings_df = pd.DataFrame(common_user_dicts)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


In [33]:
manual_embeddings_df

,calls,folds,raises,bets,checks,pre-flop_raises_user_action_sum_count,pre-flop_raises_user_action_sum_mean,pre-flop_raises_user_action_sum_sum,pre-flop_raises_user_action_sum_std,pre-flop_bets_user_action_sum_count,...,24_river_checks_user_action_sum_count,24_river_checks_user_action_sum_mean,24_river_checks_user_action_sum_sum,24_river_checks_user_action_sum_std,24_river_calls_user_action_sum_count,24_river_calls_user_action_sum_mean,24_river_calls_user_action_sum_sum,24_river_calls_user_action_sum_std,user_id,unique_id
0,0,0,0,0,0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,0,0,0,0,55vale55,55vale55_6609_#000911508468:_1
1,0,1,0,0,0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,0,0,0,0,55vale55,55vale55_6609_#000911508491:_1
2,0,2,0,0,0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,0,0,0,0,55vale55,55vale55_6613_#000911507406:_1
3,1,2,0,0,0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,0,0,0,0,55vale55,55vale55_6613_#000911507406:_5
4,2,2,0,0,0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,0,0,0,0,55vale55,55vale55_6613_#000911507406:_8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15014,0,0,0,0,0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,0,0,0,0,windbuc,windbuc_6614_#000911461531:_1
15015,0,1,0,0,0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,0,0,0,0,windbuc,windbuc_6614_#000911463100:_1
15016,0,1,1,0,0,1.0,0.06,0.06,0.0,0,...,0,0,0,0,0,0,0,0,windbuc,windbuc_6615_#000911460194:_1
15017,0,2,1,0,0,1.0,0.06,0.06,0.0,0,...,0,0,0,0,0,0,0,0,windbuc,windbuc_6615_#000911460329:_2


In [34]:
manual_embeddings_df.to_csv('manual_player_embeddings_no_data_leakege.csv', index=False)